In [446]:
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
import requests
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from typing import Optional, Dict, Any
import json
import logging
import warnings

In [447]:
load_dotenv()
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")
memory = MemorySaver()

In [448]:
# 設置日誌配置
logging.basicConfig(
    filename='proxmox_api.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")

In [449]:
class ToolRegistry:
    tools=[]
    
    @classmethod
    def register(cls,func):
        """裝飾器，用於註冊工具"""
        # 先使用 langchain 的 @tool 裝飾器
        decorated_func = tool(func)
        # 將裝飾後的函數加入到工具列表
        cls.tools.append(decorated_func)
        return decorated_func

In [450]:
class ProxmoxAPI:
    
    HOST='192.168.194.128'
    PORT=8006
    base_url=f'https://{HOST}:{PORT}/api2/json'
    headers={
    'Authorization': 'PVEAPIToken=root@pam!admin=36f55e65-9206-41b9-bcf2-d04fa6fb30db'
    }

    @classmethod
    def execute_request(cls, method: str, endpoint: str, params: Optional[Dict] = None):
        """執行 REST 請求"""
        url = f"{cls.base_url}{endpoint}"
             
        try:
            response = requests.request(
                method=method,
                url=url,
                headers=cls.headers,
                params=params,
                verify=False
            )
            
            # 檢查響應狀態，如果不是 2xx，則拋出異常
            response.raise_for_status()
            response_data = response.json()
            
            logger.info(f'API Request: {method} {url} "HTTP/1.1 {response.status_code} {response.reason}"')
            
            return response_data
            
        except Exception as e:
            # 記錄異常
            logger.error(f"API Exception: {str(e)}", exc_info=True)
            return {"error": str(e)}

@ToolRegistry.register
def excute_proxmox_api(api_schema:str):
    """
    執行 Proxmox API 請求。
    api_schema 應該是一個 JSON 字串，包含以下欄位：
    {
        "method": "GET/POST/PUT/DELETE",
        "endpoint": "/nodes/{node}/status",
        "params": {} (可選)
    }
    """
    try:
        schema = json.loads(api_schema)
        # 驗證必要欄位
        if not all(k in schema for k in ["method", "endpoint"]):
            return {"error": "缺少必要欄位"}
        
        # 執行請求
        return ProxmoxAPI.execute_request(
            method=schema["method"],
            endpoint=schema["endpoint"],
            params=schema.get("params")
        )
    except json.JSONDecodeError:
        return {"error": "無效的 JSON 格式"}
    except Exception as e:
        return {"error": str(e)}

In [451]:
system_prompt = """你是一位 Proxmox VE API 專家。你的任務是：
1. 理解用戶的需求
2. 將需求轉換為適當的 Proxmox API 請求
3. 生成正確的 API schema JSON
4. 使用 execute_proxmox_api 工具執行請求 (若操作屬於敏感操作，請暫停後續所有動作並請求用戶確認)
5. 解析並說明結果

請確保生成的 API schema 符合以下格式：
{
    "method": "GET/POST/PUT/DELETE",
    "endpoint": "/api/endpoint/path",
    "params": {} (可選)
}

請一步一步思考並執行：
1. 分析用戶需求
2. 選擇合適的 API endpoint
3. 生成 API schema
4. 執行請求
5. 解釋結果
"""

In [452]:
tools=ToolRegistry.tools
agent = create_react_agent(llm,tools,state_modifier=system_prompt,checkpointer=memory)

In [456]:
query= """
你還記得我剛剛原本提出的要求嗎?
"""
config = {"configurable": {"thread_id": "test123"}}
for s in agent.stream(
    {'messages':query},config,stream_mode='values'):
    message = s['messages'][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================


你還記得我剛剛原本提出的要求嗎?



2024-11-15 14:44:01,179 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


================================== Ai Message ==================================

抱歉，這是一個新的對話，我沒有辦法看到之前的對話內容。為了能夠更好地幫助你，請你重新告訴我你的需求是什麼？

我可以透過 Proxmox VE API 協助你完成各種操作，例如：
- 查看節點和虛擬機狀態
- 管理虛擬機（創建、啟動、停止、刪除等）
- 管理存儲和備份
- 監控系統資源
- 等等...

請告訴我你想要執行什麼操作，我會幫你一步一步完成。
